<a href="https://colab.research.google.com/github/Rahulraj31/Fake-News-LSTM/blob/main/Bi_directional_LSTM_Fake_News_Classifieripynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Fake News Classifier Using Bi- directional LSTM


Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/fake-news.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
import pandas as pd
df= pd.read_csv("/content/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


**1: unreliable**<br>

**0: reliable**

In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
# Get the Independent and Dependent Features
X=df.drop('label',axis=1)
Y=df['label']

In [8]:
X.shape,Y.shape

((18285, 4), (18285,))

In [27]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences  #for proper same size input to embedding layer, same as widow in genshim
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dense,Dropout

In [10]:
# vocabulary size
voc_size =5000

## One hot Representation


In [11]:
messages=X.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [15]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages["title"][i]) #substitute everthing with blank apart from letters a-z and A-Z in title
  
  review =review.lower() #convert to lower case
  
  review=review.split() 

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #stems words no present in stopword
  review = ' '.join(review) #combining list of words to a list of sentences
  corpus.append(review)
  

In [16]:
onehot_rep= [one_hot(words,voc_size) for words in corpus]

In [17]:
# one hot for first 5 sentences
onehot_rep[0:5]

[[3314, 1460, 1045, 43, 4665, 2380, 4751, 4895, 2960, 583],
 [1794, 4921, 1478, 1706, 39, 1735, 2764],
 [4389, 3267, 1099, 441],
 [688, 4807, 2196, 1415, 3358, 1424],
 [58, 39, 2145, 373, 1532, 4072, 39, 2354, 1442, 406]]

## Embedding Representation

In [18]:
sent_length =20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 4895 2960  583]
 [   0    0    0 ...   39 1735 2764]
 [   0    0    0 ... 3267 1099  441]
 ...
 [   0    0    0 ... 2794 4781 4843]
 [   0    0    0 ... 3316 4410 4761]
 [   0    0    0 ... 3874  464 4098]]


In [19]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 1794, 4921, 1478, 1706,   39, 1735, 2764], dtype=int32)

In [28]:
# creating model
embedding_vector_features =40   #dimwnsion of feature vector of embedding 
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(100)))  # 1 layer 100 neuron
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))


In [29]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [30]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 200)               112800    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________


In [31]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [32]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [33]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=32)


Epoch 1/10
383/383 [==============================] - 13s 11ms/step - loss: 0.2822 - accuracy: 0.8727 - val_loss: 0.2031 - val_accuracy: 0.9168
Epoch 2/10
383/383 [==============================] - 4s 9ms/step - loss: 0.1463 - accuracy: 0.9448 - val_loss: 0.2046 - val_accuracy: 0.9193
Epoch 3/10
383/383 [==============================] - 4s 9ms/step - loss: 0.1104 - accuracy: 0.9584 - val_loss: 0.1998 - val_accuracy: 0.9228
Epoch 4/10
383/383 [==============================] - 4s 10ms/step - loss: 0.0859 - accuracy: 0.9689 - val_loss: 0.2456 - val_accuracy: 0.9185
Epoch 5/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0691 - accuracy: 0.9760 - val_loss: 0.2438 - val_accuracy: 0.9138
Epoch 6/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0512 - accuracy: 0.9826 - val_loss: 0.2826 - val_accuracy: 0.9135
Epoch 7/10
383/383 [==============================] - 4s 9ms/step - loss: 0.0503 - accuracy: 0.9832 - val_loss: 0.2996 - val_accuracy: 0.9188
Epo


## Performance Metrics And Accuracy

In [34]:
y_pred=model.predict_classes(X_val)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [35]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_val,y_pred)

array([[3088,  331],
       [ 179, 2437]])

In [36]:
accuracy_score(y_val,y_pred)

0.9154929577464789

In [37]:
from sklearn.metrics import classification_report
print(classification_report(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.90      0.92      3419
           1       0.88      0.93      0.91      2616

    accuracy                           0.92      6035
   macro avg       0.91      0.92      0.91      6035
weighted avg       0.92      0.92      0.92      6035

